# Houston precipitation extremes

# Houston precipitation extremes ✏️

This notebook demonstrates extreme value analysis concepts using real
Houston precipitation data, illustrating the fundamental extrapolation
problem in climate risk assessment.

> **Purpose**
>
> This computational example supports the [Extreme Value
> Theory](../chapters/hazard/extremes.qmd) chapter by providing concrete
> data to illustrate key concepts with real Houston precipitation
> measurements.

In [1]:
using DataFrames
using Statistics
using Printf
using Random
using CSV
using Dates
using Distributions

using CairoMakie
using LaTeXStrings

CairoMakie.activate!(; type="svg")

In [1]:
#| label: load-raw-data
#| echo: false
# Load GHCN daily precipitation data from Houston area station USC00414313
# Data downloaded from: https://www.ncei.noaa.gov/pub/data/ghcn/daily/by_station/USC00414313.csv.gz

raw_data = CSV.read(joinpath(dirname(@__FILE__()), "data/USC00414313.csv"), DataFrame, header=true)
station_id = "GHCND:USC00414313"
station_name = "HOUSTON BARKER, TX US"

println("Loaded $(nrow(raw_data)) raw GHCN daily observations")
println("First few rows:")
display(first(raw_data, 3))

Loaded 70366 raw GHCN daily observations
First few rows:

This loads the raw GHCN (Global Historical Climatology Network) data
file. Each row contains weather measurements with a specific format that
we need to decode.

## Processing GHCN format

GHCN daily files use a fixed format where each row represents one
measurement at one station on one date:

In [1]:
#| label: process-ghcn
#| echo: false
# GHCN daily format: STATION, DATE, ELEMENT, VALUE, MFLAG, QFLAG, SFLAG, OBSTIME
# Rename columns for clarity
rename!(raw_data, [:station, :date_str, :element, :value, :mflag, :qflag, :sflag, :obstime])

# Filter for precipitation (PRCP) measurements only
precip_data = filter(row -> row.element == "PRCP", raw_data)

println("After filtering for precipitation (PRCP):")
println("- Total PRCP observations: $(nrow(precip_data))")
println("- Date range (raw): $(minimum(precip_data.date_str)) to $(maximum(precip_data.date_str))")

After filtering for precipitation (PRCP):
- Total PRCP observations: 25225
- Date range (raw): 19430102 to 20131130

GHCN includes many weather variables (temperature, snow, etc.), but we
only need precipitation (PRCP). The date is stored as YYYYMMDD format
and values are in tenths of millimeters.

## Converting dates and units

In [1]:
#| label: convert-format
#| echo: false
# Convert date string (YYYYMMDD) to proper Date format
precip_data.date = Date.(string.(precip_data.date_str), "yyyymmdd")

# Convert precipitation from tenths of mm to mm
precip_data.rainfall_mm = precip_data.value / 10.0

# Extract year for easier analysis
precip_data.year = year.(precip_data.date)

println("After format conversion:")
println("- Daily observations: $(nrow(precip_data))")
println("- Date range: $(minimum(precip_data.date)) to $(maximum(precip_data.date))")
println("- Years of data: $(maximum(precip_data.year) - minimum(precip_data.year) + 1)")
println("- Max daily rainfall: $(maximum(precip_data.rainfall_mm)) mm")

After format conversion:
- Daily observations: 25225
- Date range: 1943-01-02 to 2013-11-30
- Years of data: 71
- Max daily rainfall: 272.3 mm

Now we have properly formatted dates and rainfall measurements in
standard millimeter units. This represents over 70 years of daily
precipitation measurements - our “high-frequency observations.”

## Filling missing dates for complete time series

GHCN data only includes dates where measurements were recorded. For
proper time series analysis and plotting, we need to fill in missing
dates:

In [1]:
#| label: fill-missing-dates
#| echo: false
# Create complete date range from first to last observation
date_range = collect(minimum(precip_data.date):Day(1):maximum(precip_data.date))
complete_dates = DataFrame(date=date_range)

println("Date completeness:")
println("- GHCN observations: $(nrow(precip_data)) days")
println("- Complete date range: $(length(date_range)) days")
println("- Missing days: $(length(date_range) - nrow(precip_data))")

# Left join to create complete time series with missing values  
# Select only the columns we need from precip_data to avoid conflicts
precip_subset = select(precip_data, :date, :rainfall_mm)
complete_precip = leftjoin(complete_dates, precip_subset, on=:date)

# Add year column for complete dataset (overwrite any existing year column)
complete_precip.year = year.(complete_precip.date)

println("- Complete time series: $(nrow(complete_precip)) days")
println("- Missing rainfall values: $(sum(ismissing.(complete_precip.rainfall_mm)))")

Date completeness:
- GHCN observations: 25225 days
- Complete date range: 25901 days
- Missing days: 676
- Complete time series: 25901 days
- Missing rainfall values: 676

This creates a complete daily time series from
\$(minimum(precip_data.date)) to \$(maximum(precip_data.date)) with
`missing` values for days without measurements. Missing values are
common in historical weather data due to equipment issues, observer
absence, or data quality problems.

## Daily precipitation time series

Now let’s visualize the daily precipitation data to see the
“high-frequency observations” mentioned in our extrapolation problem:

In [1]:
#| label: houston-data
#| echo: false
nyears = length(unique(complete_precip.year))

fig = Figure(size=(800, 400))
ax = Axis(fig[1, 1],
    title="Daily Precipitation: $(station_name) ($nyears years)",
    ylabel="Daily Rainfall (mm)")

# Plot daily precipitation as lines (missing values will create gaps)
lines!(ax, complete_precip.date, complete_precip.rainfall_mm,
    color=:steelblue, linewidth=1)

for threshold in [300, 150]
    n_exceed = sum(skipmissing(complete_precip.rainfall_mm .> threshold))
    hlines!(ax, [threshold], linestyle=:dash, label="$threshold ($n_exceed exceedances)", linewidth=3)
end

axislegend(ax, position=:lt)
fig

Or a subset

In [1]:
#| label: houston-subset
#| echo: false
precip_plt = complete_precip[Date(1999, 1, 1).<=complete_precip.date.<=Date(1999, 12, 31), :]

fig = Figure(size=(800, 400))
ax = Axis(fig[1, 1],
    title="Daily Precipitation: $(station_name) (1999)",
    ylabel="Daily Rainfall (mm)")

# Plot daily precipitation as lines (missing values will create gaps)
lines!(ax, precip_plt.date, precip_plt.rainfall_mm,
    color=:steelblue, linewidth=1)

fig

## Shape parameter implications

The shape parameter ξ in extreme value distributions fundamentally
determines how heavy the tail is, which directly affects our
extrapolation to rare events. Let’s visualize how different shape
parameter values lead to dramatically different tail behavior:

In [1]:
#| label: shape-parameter-comparison
#| echo: false
# Set up GEV parameters with same location/scale but different shape
μ = 100.0  # location parameter (typical annual maximum rainfall, mm)
σ = 20.0   # scale parameter (variability)
ξ_values = [-0.2, 0.0, 0.2]  # shape parameters to compare
colors = [:red, :blue, :green]
labels = [L"\xi = -0.2" * " (Weibull)", L"\xi = 0.0" * " (Gumbel)", L"\xi = 0.2" * " (Fréchet)"]

# Create range of values for plotting
x_range = 60:0.5:300

fig = Figure(size=(800, 600))

# Plot PDFs
ax1 = Axis(fig[1, 1],
    title="GEV Probability Density Functions",
    xlabel=L"Annual Maximum Rainfall (mm)",
    ylabel=L"Density")

for (i, ξ) in enumerate(ξ_values)
    if ξ == 0.0
        # Gumbel distribution (ξ = 0 limit)
        dist = Gumbel(μ, σ)
    else
        # GeneralizedExtremeValue(μ, σ, ξ)
        dist = GeneralizedExtremeValue(μ, σ, ξ)
    end

    y_vals = pdf.(dist, x_range)
    lines!(ax1, x_range, y_vals, color=colors[i], linewidth=3, label=labels[i])
end

axislegend(ax1, position=:rt)

# Plot survival functions (1 - CDF) on log scale to highlight tail behavior
ax2 = Axis(fig[2, 1],
    title="GEV Survival Functions (Tail Behavior)",
    xlabel=L"Annual Maximum Rainfall (mm)",
    ylabel=L"Pr(X > x)",
    yscale=log10)

for (i, ξ) in enumerate(ξ_values)
    if ξ == 0.0
        dist = Gumbel(μ, σ)
    else
        dist = GeneralizedExtremeValue(μ, σ, ξ)
    end

    # Only plot where survival function is positive
    survival_vals = 1 .- cdf.(dist, x_range)
    # Filter out zero/negative values for log scale
    valid_indices = survival_vals .> 1e-10
    x_valid = x_range[valid_indices]
    y_valid = survival_vals[valid_indices]

    lines!(ax2, x_valid, y_valid, color=colors[i], linewidth=3, label=labels[i])
end

axislegend(ax2, position=:rt)
fig

The key insight: **the shape parameter ξ controls how rapidly
probabilities decrease in the tail**.

For our Houston rainfall example, this has practical implications:

In [1]:
#| label: return-level-comparison
#| echo: false
# Calculate return levels for different return periods
return_periods = [10, 50, 100, 500, 1000]

# Create a table comparing return levels
results_df = DataFrame(
    return_period=Int[],
    xi_neg02=Float64[],
    xi_zero=Float64[],
    xi_pos02=Float64[],
)

for T in return_periods
    p = 1.0 / T  # exceedance probability

    # Calculate return levels for each shape parameter
    levels = Float64[]

    for ξ in ξ_values
        if ξ == 0.0
            # Gumbel case: x_T = μ - σ * ln(-ln(1-p))
            x_T = μ - σ * log(-log(1 - p))
        else
            # GEV case: x_T = μ - (σ/ξ) * [1 - (-ln(1-p))^(-ξ)]
            x_T = μ - (σ / ξ) * (1 - (-log(1 - p))^(-ξ))
        end
        push!(levels, x_T)
    end

    push!(results_df, (T, levels[1], levels[2], levels[3]))
end

println("Return Level Comparison (mm of daily rainfall):")
println("=" ^ 60)
Printf.@printf("%-15s %-12s %-12s %-12s\n", "Return Period", "ξ = -0.2", "ξ = 0.0", "ξ = 0.2")
println("-" ^ 60)

for row in eachrow(results_df)
    Printf.@printf("%-15s %-12.1f %-12.1f %-12.1f\n",
        "$(row.return_period)-year", row.xi_neg02, row.xi_zero, row.xi_pos02)
end

# Calculate percentage differences from Gumbel case
println("\nPercentage difference from Gumbel (ξ = 0):")
println("=" ^ 50)
Printf.@printf("%-15s %-12s %-12s\n", "Return Period", "ξ = -0.2", "ξ = 0.2")
println("-" ^ 50)

for row in eachrow(results_df)
    diff_neg = ((row.xi_neg02 - row.xi_zero) / row.xi_zero) * 100
    diff_pos = ((row.xi_pos02 - row.xi_zero) / row.xi_zero) * 100
    Printf.@printf("%-15s %-12.1f%% %-12.1f%%\n",
        "$(row.return_period)-year", diff_neg, diff_pos)
end

Return Level Comparison (mm of daily rainfall):
Return Period   ξ = -0.2     ξ = 0.0      ξ = 0.2     
------------------------------------------------------------
10-year         136.2        145.0        156.8       
50-year         154.2        178.0        218.2       
100-year        160.1        192.0        250.9       
500-year        171.1        224.3        346.5       
1000-year       174.9        238.1        398.1       

Percentage difference from Gumbel (ξ = 0):
Return Period   ξ = -0.2     ξ = 0.2     
--------------------------------------------------
10-year         -6.0        % 8.2         %
50-year         -13.4       % 22.6        %
100-year        -16.6       % 30.7        %
500-year        -23.7       % 54.5        %
1000-year       -26.6       % 67.2        %